
# 🚗 AWS DynamoDB – Car Rentals Lab (Using boto3.resource)

This lab demonstrates how to use **`boto3.resource()`** to interact with **DynamoDB** through a real-world **Car Rentals** example.

We will create a table, insert data, query, update, delete, and finally clean up the resources — all using the high-level **boto3.resource()** interface.



## ✅ Prerequisites

- AWS account and credentials configured (`aws configure`)
- IAM user or role with `AmazonDynamoDBFullAccess`
- Python 3.7+ and `boto3` installed

```bash
pip install boto3
```



## ⚙️ Step 1 – Create DynamoDB Table

We’ll create a table named **CarRentals_Resource** with:
- **Partition key:** `RentalID` (String)
- **Sort key:** `CustomerID` (String)


In [ ]:

import boto3
from boto3.dynamodb.conditions import Key, Attr
import time

# Initialize DynamoDB resource
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

table_name = 'CarRentals_Resource'

# Create table
table = dynamodb.create_table(
    TableName=table_name,
    KeySchema=[
        {'AttributeName': 'RentalID', 'KeyType': 'HASH'},  # Partition key
        {'AttributeName': 'CustomerID', 'KeyType': 'RANGE'}  # Sort key
    ],
    AttributeDefinitions=[
        {'AttributeName': 'RentalID', 'AttributeType': 'S'},
        {'AttributeName': 'CustomerID', 'AttributeType': 'S'}
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

print("Creating table...")
table.wait_until_exists()
print(f"✅ Table '{table_name}' is active now.")



## 🚘 Step 2 – Insert Car Rental Records

We’ll insert three records showing DynamoDB’s flexibility in handling different attributes.


In [ ]:

table = dynamodb.Table(table_name)

rental_records = [
    {
        'RentalID': 'RNT2001',
        'CustomerID': 'CUST201',
        'CarModel': 'Tata Nexon',
        'RentalDays': 3,
        'PickupCity': 'Bangalore',
        'DropoffCity': 'Coimbatore',
        'PaymentMode': 'UPI',
        'TotalAmount': 8500
    },
    {
        'RentalID': 'RNT2002',
        'CustomerID': 'CUST202',
        'CarModel': 'Maruti Baleno',
        'RentalDays': 2,
        'PickupCity': 'Chennai',
        'Extras': {'Insurance': 'Yes', 'GPS': 'No'},
        'Discount': 15
    },
    {
        'RentalID': 'RNT2003',
        'CustomerID': 'CUST203',
        'CarModel': 'Honda Amaze',
        'RentalDays': 5,
        'PickupCity': 'Hyderabad',
        'DropoffCity': 'Hyderabad',
        'Extras': ['MusicSystem', 'ChildSeat'],
        'TotalAmount': 11200
    }
]

for record in rental_records:
    table.put_item(Item=record)

print("✅ Inserted 3 car rental records successfully.")



## 🔍 Step 3 – Scan and Query the Table

Retrieve all rentals or filter by attributes.


In [ ]:

# Scan all records
response = table.scan()
print("All Car Rental Records:")
for item in response['Items']:
    print(item)


In [ ]:

# Query by RentalID
response = table.query(
    KeyConditionExpression=Key('RentalID').eq('RNT2001')
)

print("Query Result for RentalID = RNT2001:")
for item in response['Items']:
    print(item)



## ✏️ Step 4 – Update an Item

Update the `TotalAmount` or add a new attribute such as `DiscountApplied`.


In [ ]:

response = table.update_item(
    Key={
        'RentalID': 'RNT2001',
        'CustomerID': 'CUST201'
    },
    UpdateExpression='SET TotalAmount = :val1, DiscountApplied = :val2',
    ExpressionAttributeValues={
        ':val1': 8800,
        ':val2': True
    },
    ReturnValues='UPDATED_NEW'
)

print("✅ Updated Item:", response['Attributes'])



## ❌ Step 5 – Delete an Item

Remove one car rental record.


In [ ]:

response = table.delete_item(
    Key={
        'RentalID': 'RNT2002',
        'CustomerID': 'CUST202'
    }
)

print("✅ Deleted record RNT2002 successfully.")



## 🧹 Step 6 – Cleanup

Delete the table once validation is complete.


In [ ]:

table.delete()
print(f"🗑️ Deleting table '{table_name}'...")

# Wait until the table no longer exists
while True:
    try:
        dynamodb.Table(table_name).load()
        print("⏳ Waiting for table deletion...")
        time.sleep(5)
    except dynamodb.meta.client.exceptions.ResourceNotFoundException:
        print(f"✅ Table '{table_name}' deleted successfully.")
        break
